In [42]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pprint
import json
import urllib
import requests

# Get Covid data from api.covidtracking.com
1. suppose 'data' is the retrieved json, then it has keys ['links', 'meta', 'data']<br>
2. For us/daily, data['data'] stores daily number of cases, with keys ['date', 'states', 'cases', 'testing', 'outcomes']<br>
['date'] is the date in YYYY-MM-DD<br>
['state'] is the number of states<br>
['cases'] has a single key ['total'] with ['value'] (raw number) and ['calculated'] (some statistics)<br>
['testing'] is structured the same way as ['cases']<br>
['outcomes']<br>
3. For state/daily, data['data'] has keys ['date', 'state', 'meta', 'cases', 'tests', 'outcomes']

In [106]:
def get_state_codes():
    l = 'https://api.covidtracking.com/v2/states.json'
    with urllib.request.urlopen(l) as url:
        link_json = json.load(url)
    pd.DataFrame(link_json['data']).to_csv('./data/states.csv')

In [128]:
def retrieve_json(link):
    try:
        with urllib.request.urlopen(link) as url:
            link_json = json.load(url)
        filename = './data/' + link.split('v2/')[1].replace('/', '_')
        with open(filename, 'w') as f:
            json.dump(link_json, f)
        print('saved ' + filename)
        if filename != './data/us_daily.json':
            covid_files.append(filename)
    except:
        print('retrieval failed ' + link)

In [113]:
def get_all_json():
    global state_codes
    covidtracking_api = 'https://api.covidtracking.com/v2/'
    
    retrieve_json(covidtracking_api+'us/daily.json')
    
    states_codes = pd.read_csv('./data/states.csv')['state_code'].str.lower()
    for state in states_codes:
        retrieve_json(covidtracking_api + 'states/' + state + '/daily/simple.json')

In [169]:
def clean_df_from_json(filename):
    with open(filename, 'r') as f:
        temp = json.load(f)
    temp = pd.DataFrame(temp['data'])
    
    if filename != './data/us_daily.json':
        temp['cases_total'] = temp['cases'].apply(lambda x: x['total'])
        temp['cases_confirmed'] = temp['cases'].apply(lambda x: x['confirmed'])
        temp['cases_probable'] = temp['cases'].apply(lambda x: x['probable'])
        temp['tests_total'] = temp['tests'].apply(lambda x: x['pcr']['specimens']['total'])
        temp['tests_positive'] = temp['tests'].apply(lambda x: x['pcr']['specimens']['positive'])
        temp = temp.drop(['cases', 'tests'], axis=1)
        temp.to_csv(filename.replace('json', 'csv'))
        print('saved ' + filename.replace('json', 'csv'))
    else:
        temp['cases_total'] = temp['cases'].apply(lambda x: x['total']['value'])
        temp['cases_percent'] = temp['cases'].apply(lambda x: x['total']['calculated']['population_percent'])
        temp['cases_increase'] = temp['cases'].apply(lambda x: x['total']['calculated']['change_from_prior_day'])
        temp['cases_7d_change'] = temp['cases'].apply(lambda x: x['total']['calculated']['seven_day_change_percent'])
        temp['testing_total'] = temp['testing'].apply(lambda x: x['total']['value'])
        temp['testing_percent'] = temp['testing'].apply(lambda x: x['total']['calculated']['population_percent'])
        temp['testing_increase'] = temp['testing'].apply(lambda x: x['total']['calculated']['change_from_prior_day'])
        temp['testing_7d_change'] = temp['testing'].apply(lambda x: x['total']['calculated']['seven_day_change_percent'])
        temp = temp.drop(['cases', 'testing'], axis=1)
        temp.to_csv('./data/us_daily.csv')
        print('saved us_daily.csv')

In [170]:
def clean_all_df():
    clean_df_from_json('./data/us_daily.json')
    for file in covid_files:
        clean_df_from_json(file)

In [107]:
get_state_codes()

In [93]:
covid_files = []
get_all_json()

saved states_al_daily_simple.json
saved states_ak_daily_simple.json
saved states_az_daily_simple.json
saved states_ar_daily_simple.json
saved states_ca_daily_simple.json
saved states_co_daily_simple.json
saved states_ct_daily_simple.json
saved states_de_daily_simple.json
saved states_dc_daily_simple.json
saved states_fl_daily_simple.json
saved states_ga_daily_simple.json
saved states_hi_daily_simple.json
saved states_id_daily_simple.json
saved states_il_daily_simple.json
saved states_in_daily_simple.json
saved states_ia_daily_simple.json
saved states_ks_daily_simple.json
saved states_ky_daily_simple.json
saved states_la_daily_simple.json
saved states_me_daily_simple.json
saved states_md_daily_simple.json
saved states_ma_daily_simple.json
saved states_mi_daily_simple.json
saved states_mn_daily_simple.json
saved states_ms_daily_simple.json
saved states_mo_daily_simple.json
saved states_mt_daily_simple.json
saved states_ne_daily_simple.json
saved states_nv_daily_simple.json
saved states_n

In [171]:
clean_all_df()

saved us_daily.csv
saved states_al_daily_simple.csv
saved states_ak_daily_simple.csv
saved states_az_daily_simple.csv
saved states_ar_daily_simple.csv
saved states_ca_daily_simple.csv
saved states_co_daily_simple.csv
saved states_ct_daily_simple.csv
saved states_de_daily_simple.csv
saved states_dc_daily_simple.csv
saved states_fl_daily_simple.csv
saved states_ga_daily_simple.csv
saved states_hi_daily_simple.csv
saved states_id_daily_simple.csv
saved states_il_daily_simple.csv
saved states_in_daily_simple.csv
saved states_ia_daily_simple.csv
saved states_ks_daily_simple.csv
saved states_ky_daily_simple.csv
saved states_la_daily_simple.csv
saved states_me_daily_simple.csv
saved states_md_daily_simple.csv
saved states_ma_daily_simple.csv
saved states_mi_daily_simple.csv
saved states_mn_daily_simple.csv
saved states_ms_daily_simple.csv
saved states_mo_daily_simple.csv
saved states_mt_daily_simple.csv
saved states_ne_daily_simple.csv
saved states_nv_daily_simple.csv
saved states_nh_daily_si

# Get Covid data (of March 10) from worldometers

In [6]:
wdm = 'https://www.worldometers.info/coronavirus/country/us/'
wdm_page = requests.get(wdm)

In [12]:
df = pd.read_html(wdm_page.content)[1]
df.columns.to_list() # check weird column names

['#',
 'USAState',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/ 1M pop',
 'Population',
 'Projections']

In [16]:
# change weird column names
df = df.rename(columns={'Tot\xa0Cases/1M pop': 'TotCasesPer1MPop', 
                        'Deaths/1M pop':'DeathsPer1MPop', 'Tests/ 1M pop': 'TestsPer1MPop'})
df.to_csv('./data/us_0310.csv')

# Get weather data from api.weather.gov (abandoned)

1. county -> severals zones -> several stations
2. county observations are completely empty
3. county observationStations are also empty<br>
Abandoned because the api does not provide enough data

In [100]:
# the county_stations is always empty as returned by the api, so we have another function
# get_state_stations
def get_state_counties(state):
    state_abbr = states[states['name']==state]['state_code'].values[0]
    t = f'https://api.weather.gov/zones?area={state_abbr}&type=county'
    with urllib.request.urlopen(t) as url:
            k = url.read()
    print(t + ' api call is successful')
    j = json.loads(k)['features']
    
    county_links = [item['id'] for item in j]
    county_ids = [item['properties']['id'] for item in j]
    county_names = [item['properties']['name'] for item in j]
    county_states = [item['properties']['state'] for item in j]
    county_stations = [item['properties']['observationStations'] for item in j] # empty
    
    df = pd.DataFrame({'link': county_links, 'id': county_ids, 'name': county_names,
                      'state_code': county_states, 'stations': county_stations})
    df['state'] = [state]*len(j)
    return df

In [23]:
# the api does not show any stations for a given county, so we have to get all stations in
# a state and match them to their counties
def get_state_stations(state):
    state_abbr = states[states['name']==state]['state_code'].values[0]
    t = f'https://api.weather.gov/stations?state={state_abbr}'
    with urllib.request.urlopen(t) as url:
            k = url.read()
    print(t + ' api call is successful')
    j = json.loads(k)['features']
    
    station_forecasts = []
    for i, item in enumerate(j):
        if 'forecast' in item['properties'].keys():
            station_forecasts.append(item['properties']['forecast'])
        else:
            station_forecasts.append(np.nan)

    station_counties = []
    for i, item in enumerate(j):
        if 'county' in item['properties'].keys():
            station_counties.append(item['properties']['county'].split('/')[-1])
        else:
            station_counties.append(np.nan)
    
    df = pd.DataFrame({'forecast': station_forecasts, 'county': station_counties})
    df = df.dropna(axis=0)
    df = df.drop_duplicates(ignore_index=True)
    
    return df

In [122]:
def get_station_county_pair(state):
    counties_of_state = get_state_counties(state)
    stations_of_state = get_state_stations(state)
    df = pd.merge(cali, cali_station, left_on='id', right_on='county', 
                  suffixes=['_county', '_station'])
    df = df.drop('county', axis=1)
    df.to_csv(f'./data/{state}_station_county_pair.csv')

In [ ]:
def get_state_weather(state):
    station_county_map = pd.read_csv(f'./data/{state}_station_county_pair.csv', index_col=0)

In [131]:
states = pd.read_csv('./data/states.csv', index_col=0)

In [123]:
get_station_county_pair('California')

https://api.weather.gov/zones?area=CA&type=county api call is successful
https://api.weather.gov/stations?state=CA api call is successful


# Get weather data from NOAA

In [191]:
datatypes = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?limit=1000'
datatypes_r = requests.get(datatypes, headers={'token': token})

temp = json.loads(datatypes_r.text)['results']
mindate = [item['mindate'] for item in temp]
maxdate = [item['maxdate'] for item in temp]
name = [item['name'] for item in temp]
_id = [item['id'] for item in temp]
df = pd.DataFrame({'mindate': mindate, 'maxdate': maxdate, 'name': name, 'id': _id})
df.to_csv('noaa_datatypes.csv')

In [335]:
def get_county_weather(state):
    county_fips = us_counties[us_counties['state']==state]['fips'].unique()
    params = '&startdate=2020-01-01&enddate=2021-03-08&datatypeid=TAVG&limit=1000'
    total = pd.DataFrame()
    print(f'{len(county_fips)} counties')
    for fips in county_fips:
        link = noaa_api + f'&locationid=FIPS:{int(fips)}' + params
        r = requests.get(l, headers={'token': token})
        if r.status_code != 200:
            print(fips, 'failed')
            continue
        print(fips, 'connected', end=' ')
        d = json.loads(r.text)['results']
        dates = [item['date'] for item in d]
        values = [item['value'] for item in d]
        df = pd.DataFrame({'TAVG': values, 'date': dates})
        df['fips'] = [fips] * len(d)

        mean_tavg = df.groupby(['fips', 'date'])['TAVG'].mean().values
        df = df.drop('TAVG', axis=1)
        df = df.drop_duplicates(ignore_index=True)
        df['mean_temp'] = mean_tavg

        total = pd.concat((total, df))
    total.to_csv(f'./weather_data/{state}_weather.csv')
    return total

In [332]:
us_counties = pd.read_csv('./data/us-counties.csv')
us_counties['date'] = pd.to_datetime(us_counties['date'])
us_counties = us_counties.dropna()

In [216]:
us_counties.columns

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')

In [273]:
noaa_api = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND'
token = 'rilsfDWJGSwodWyqrUaOsREORAwwngJa'

In [333]:
county_weather = get_county_weather('California')

58 counties
6059.0 connected
6037.0 connected
6085.0 connected
6075.0 connected
6073.0 connected
6023.0 connected
6067.0 connected
6095.0 connected
6041.0 connected
6055.0 connected
6097.0 connected
6001.0 connected
6061.0 connected
6081.0 connected
6013.0 connected
6113.0 connected
6019.0 connected
6039.0 connected
6065.0 connected
6087.0 connected
6089.0 connected
6077.0 connected
6111.0 connected
6099.0 connected
6107.0 connected
6069.0 connected
6079.0 connected
6071.0 connected
6083.0 connected
6057.0 connected
6029.0 connected
6053.0 connected
6045.0 connected
6005.0 connected
6025.0 connected
6007.0 connected
6017.0 connected
6093.0 connected
6115.0 connected
6009.0 connected
6047.0 connected
6051.0 connected
6027.0 connected
6101.0 connected
6011.0 connected
6031.0 connected
6021.0 connected
6109.0 connected
6003.0 connected
6063.0 connected
6015.0 connected
6103.0 connected
6033.0 connected
6043.0 connected
6105.0 connected
6091.0 connected
6035.0 connected
6049.0 connected


In [277]:
l = noaa_api+'&locationid=FIPS:53061&startdate=2020-01-01&enddate=2020-01-03&datatypeid=TAVG&limit=1000'
r = requests.get(l, headers={'token': token})
#load the api response as a json
print(r)
d = json.loads(r.text)
d.keys()

<Response [200]>


dict_keys(['metadata', 'results'])

In [278]:
d['metadata']

{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}

In [289]:
d['results'][5]

{'date': '2020-01-03T00:00:00',
 'datatype': 'TAVG',
 'station': 'GHCND:USS0021B48S',
 'attributes': ',,T,',
 'value': 35}

In [247]:
len(np.unique([item['date'] for item in d['results']]))

366

In [304]:
dates = [item['date'] for item in d['results']]
values = [item['value'] for item in d['results']]
df = pd.DataFrame({'TAVG': values, 'date': dates})
df['fips'] = [53061] * len(d['results'])

In [305]:
m = df.groupby(['fips', 'date'])['TAVG'].mean().values

In [306]:
df = df.drop('TAVG', axis=1)
df = df.drop_duplicates(ignore_index=True)
df['mean_temp'] = m
df

,date,fips,mean_temp
0,2020-01-01T00:00:00,53061,25.0
1,2020-01-02T00:00:00,53061,-6.0
2,2020-01-03T00:00:00,53061,41.5


In [308]:
total = pd.DataFrame()
pd.concat((total, df, df))

,date,fips,mean_temp
0,2020-01-01T00:00:00,53061,25.0
1,2020-01-02T00:00:00,53061,-6.0
2,2020-01-03T00:00:00,53061,41.5
0,2020-01-01T00:00:00,53061,25.0
1,2020-01-02T00:00:00,53061,-6.0
2,2020-01-03T00:00:00,53061,41.5
